# Include libraries

In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sksurv.column import encode_categorical

## dataset loading...

In [2]:
surv_data = pd.read_csv(os.path.join('..', '..', 'data', '(v4)STROKE_VITAL_SIGN_missForest.csv'))

In [3]:
surv_data = surv_data[surv_data['CVDeath'] != 9999]

## data preprocessing

In [4]:
from sklearn import preprocessing
from sklearn_pandas import DataFrameMapper

def data_processing(data_df):
    data_df_x = data_df.drop(['LOC', 'UID', 'Mortality', 'SurvivalDays', 'CVDeath', 'SurvivalMonths',
                              'admission_date', 'discharge_date', 'death_date'], axis=1)

    data_df_y = data_df[['CVDeath', 'SurvivalDays']]

    X_temp = data_df_x[(data_df.LOC == '3') | (data_df.LOC == '2') | (data_df.LOC == '6')]
    y_temp = data_df_y[(data_df.LOC == '3') | (data_df.LOC == '2') | (data_df.LOC == '6')]
    X_df_train, X_df_val, y_df_train, y_df_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=369)

    X_df_test_kao = data_df_x[data_df.LOC == '8']
    y_df_test_kao = data_df_y[data_df.LOC == '8']

    categorical_columns = ['Sex', 'AF', 'DM', 'HTN', 'Dyslipidemia', 'CHF', 'Smoking',
                           'Cancer.before.adm']
    
    numerical_columns = np.setdiff1d(data_df_x.columns, categorical_columns).tolist()

    categorical_ix = [data_df_x.columns.get_loc(col) for col in categorical_columns]
    numerical_ix =  np.setdiff1d(list(range(0, len(data_df_x.columns))), categorical_ix).tolist()

    scaler = preprocessing.StandardScaler()

    standardize = [([col], scaler) for col in numerical_columns]
    leave = [(col, None) for col in categorical_columns]

    x_mapper = DataFrameMapper(standardize + leave)

    X_df_train = pd.DataFrame(data=x_mapper.fit_transform(X_df_train),
                              columns=numerical_columns+categorical_columns,
                              index=X_df_train.index)

    X_df_val = pd.DataFrame(data=x_mapper.fit_transform(X_df_val),
                            columns=numerical_columns+categorical_columns,
                            index=X_df_val.index)

    X_df_test_kao = pd.DataFrame(data=x_mapper.fit_transform(X_df_test_kao),
                                 columns=numerical_columns+categorical_columns,
                                 index=X_df_test_kao.index)
    # since those categorical variable are all boolen... no need to one-hot them
    # https://stackoverflow.com/questions/43515877/should-binary-features-be-one-hot-encoded
    
#     X_df_train = encode_categorical(X_df_train, columns=categorical_columns)
#     X_df_val = encode_categorical(X_df_val, columns=categorical_columns)
#     X_df_test_kao = encode_categorical(X_df_test_kao, columns=categorical_columns)
    
    return X_df_train, X_df_val, y_df_train, y_df_val, X_df_test_kao, y_df_test_kao


get_target = lambda df: (df['SurvivalDays'].values, df['CVDeath'].values)

In [5]:
X_train, X_val, y_train, y_val, X_test_kao, y_test_kao = data_processing(surv_data)
# for DLs
X_test_kao = X_test_kao.values.astype('float32')
durations_test_kao, events_test_kao = get_target(y_test_kao)
# for RSF
y_train_cox =  y_train.copy()
y_train_cox.loc[:, 'CVDeath'] = y_train_cox['CVDeath'].astype(bool)
y_train_cox = np.array(list(y_train_cox.to_records(index=False)))
y_test_kao_cox =  y_test_kao.copy()
y_test_kao_cox.loc[:, 'CVDeath'] = y_test_kao_cox['CVDeath'].astype(bool)
y_test_kao_cox = np.array(list(y_test_kao_cox.to_records(index=False)))

In [6]:
X_train.head()

,ALT,Age,CHOL,CKD,DBPCV.G,HRCV.G,Mean.PP.G,Mean.SBP.G,MeanDBP.G,MeanHR.G,...,eGFR,eNIHSS,Sex,AF,DM,HTN,Dyslipidemia,CHF,Smoking,Cancer.before.adm
13943,0.361083,0.222219,1.115646,-1.210555,-0.305294,2.102906,-0.770882,-0.334212,-0.105648,-0.811634,...,0.971321,-0.535316,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
7658,-0.058095,1.139428,-1.036461,0.079579,-0.305294,2.102906,0.555585,1.065777,0.795004,0.085422,...,-0.450463,-0.535316,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
15136,-0.351520,-0.542121,0.513982,0.079579,-0.305294,-1.040582,-0.770882,-1.034206,-0.105648,-0.811634,...,0.311627,-0.535316,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
902,0.109576,-2.147237,0.351995,-1.210555,-0.305294,0.007248,-0.107648,1.065777,1.695656,0.085422,...,0.453555,0.873945,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
12929,0.319165,-0.007083,-0.388515,-1.210555,-0.305294,0.007248,-0.770882,0.365782,0.795004,0.085422,...,0.899444,-0.535316,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


## Cox

In [7]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
estimator = CoxPHSurvivalAnalysis()
estimator.fit(X_train, y_train_cox)
print(estimator.score(X_test_kao, y_test_kao_cox))

0.816763764296244


# DL setting

In [8]:
import torch
import torchtuples as tt
from pycox.evaluation import EvalSurv

## Deepsur

In [9]:
from pycox.models import CoxPH

def Deepsur(Xtrain, Xval, Ytrain, Yval):
    # preprocessing data
    Xtrain = Xtrain.values.astype('float32')
    Xval = Xval.values.astype('float32')
    Ytrain = get_target(Ytrain)
    Yval = get_target(Yval)
    val = Xval, Yval
    
    # parameters
    in_features = Xtrain.shape[1]
    num_nodes = [48, 48]
    out_features = 1
    batch_norm = True
    dropout = 0.1
    output_bias = False
    batch_size = 128
    if Xtrain.shape[0]%batch_size == 1:
        batch_size = batch_size - 1
    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping(patience=50)]
    verbose = False
    
    # network
    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                                  dropout, output_bias=output_bias)
    model = CoxPH(net, tt.optim.Adam)
    
    # train
    lrfinder = model.lr_finder(Xtrain, Ytrain, batch_size, tolerance=10)
    #best_lr = lrfinder.get_best_lr()
    #_ = lrfinder.plot()
    model.optimizer.set_lr(0.01)
    log = model.fit(Xtrain, Ytrain, batch_size, epochs, callbacks, verbose,
                    val_data=val, val_batch_size=batch_size)
    return model
    

In [10]:
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_sur = Deepsur(X_train, X_val, y_train, y_val)

In [11]:
# prediction
_ = deep_sur.compute_baseline_hazards()
surv_kao = deep_sur.predict_surv_df(X_test_kao)
# evaluation
ev_kao = EvalSurv(surv_kao, durations_test_kao, events_test_kao, censor_surv='km')
print('Kao C-index = %.3f' %(ev_kao.concordance_td()))

Kao C-index = 0.826


## RandomSurvivalForest

In [12]:
from sksurv.ensemble import RandomSurvivalForest

rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=369)

rsf.fit(X_train, y_train_cox)
print('Kao C-index = %.3f' %(rsf.score(X_test_kao, y_test_kao_cox)))

Kao C-index = 0.822


In [13]:
# Predicted risk scores
import eli5
from eli5.sklearn import PermutationImportance
feature_names = X_train.columns.values
perm = PermutationImportance(rsf, n_iter=5, random_state=369)
perm.fit(X_test_kao, y_test_kao_cox)
eli5.show_weights(perm, feature_names=feature_names)

Weight,Feature
0.0841 ± 0.0055,eNIHSS
0.0348 ± 0.0052,Age
0.0169 ± 0.0009,eGFR
0.0064 ± 0.0035,MeanHR.G
0.0032 ± 0.0018,MeanRR.G
0.0024 ± 0.0028,AF
0.0024 ± 0.0007,SBPCV.G
0.0021 ± 0.0018,TG
0.0018 ± 0.0023,DBPCV.G
0.0016 ± 0.0018,CHOL


In [14]:
eli5.format_as_dataframe(eli5.explain_weights(perm, feature_names=feature_names, top=100)).to_csv('rfs_FS_cvd.csv', index=False)

## DeepHit

In [15]:
# from pycox.models import DeepHitSingle

# def Deephit(Xtrain, Xval, Ytrain, Yval):
#     # preprocessing data
#     Xtrain = Xtrain.values.astype('float32')
#     Xval = Xval.values.astype('float32')
    
#     # label transforms
#     num_durations = 96
#     labtrans = DeepHitSingle.label_transform(num_durations)
#     Ytrain = labtrans.fit_transform(*get_target(Ytrain))
#     Yval = labtrans.transform(*get_target(Yval))
#     train = (Xtrain, Ytrain)
#     val = (Xval, Yval)
    
#     # parameters
#     in_features = Xtrain.shape[1]
#     num_nodes = [64, 64]
#     out_features = labtrans.out_features
#     batch_norm = True
#     dropout = 0.1
#     output_bias = False
#     batch_size = 256
#     if Xtrain.shape[0]%batch_size == 1:
#         batch_size = batch_size - 1
#     epochs = 100
#     callbacks = [tt.callbacks.EarlyStopping(patience=20)]
#     verbose = False
    
#     # network
#     net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
#                                   dropout, output_bias=output_bias)
    
#     model = DeepHitSingle(net, tt.optim.Adam, alpha=0.1, sigma=0.1, duration_index=labtrans.cuts)

#     lrfinder = model.lr_finder(Xtrain, Ytrain, batch_size, tolerance=10)
#     #best_lr = lrfinder.get_best_lr()

#     model.optimizer.set_lr(0.01)
    
#     log = model.fit(Xtrain, Ytrain, batch_size, epochs, callbacks, verbose,
#                     val_data=val, val_batch_size=batch_size)
    
#     return model

In [16]:
# # Seed
# seed = 369
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# np.random.seed(seed)
# random.seed(seed)
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

# deep_hit = Deephit(X_train, X_val, y_train, y_val)

In [17]:
# # prediction
# hit_surv = deep_hit.predict_surv_df(X_test_kao)
# # evaluation
# ev_kao = EvalSurv(hit_surv, durations_test_kao, events_test_kao, censor_surv='km')
# print('Kao C-index = %.3f' %(ev_kao.concordance_td()))